In [ ]:
%matplotlib inline
import numpy as np 
import cpmpy as cp
from cpmpy.solvers import CPM_ortools
import torch 
import torch.nn as nn
import os
from collections import OrderedDict
import matplotlib.pyplot as plt
from PIL import Image

## Recap: Solving a Sudoku with CPMpy

A previous notebook covered ways to model the basic Sudoku problem in CP and solve it, using CPMpy. 



In [ ]:
def get_sudoku_model(n=9):
    b = np.sqrt(n).astype(int)
    cells = cp.IntVar(1, n, shape=(n,n))

    # plain sudoku model
    m = cp.Model(
        [cp.alldifferent(row) for row in cells],
        [cp.alldifferent(col) for col in cells.T],
        [cp.alldifferent(cells[i:i + b, j:j + b])
            for i in range(0, n, b) for j in range(0, n, b)],
    )
    return {
        'model':m,
        'variables':cells
    }

def solve_sudoku(model, dvars, instance):
    # use another object for solving
    newmodel = cp.Model(model.constraints) 
    # set given clues
    newmodel += cp.all(instance[instance>0] == dvars[instance>0])
    if newmodel.solve():
        results = {
            'runtime':np.asarray(newmodel.cpm_status.runtime),
            'solution':dvars.value(),
        }
    else:
        results = {
            'solution':np.full_like(dvars.value(), np.nan)
        }
    results['status'] = np.asarray(newmodel.cpm_status.exitstatus.value)
    return results


## Visual Sudoku

![](data/visual_sudoku/img/059.jpg) 

In this variant, the input is now a image of a cropped Sudoku grid, sample from the [Sudoku Assistant App](https://visualsudoku.cs.kuleuven.be/).

Solving a visual Sudoku instance requires an approach combining Machine Learning and Constraint Solving. Many of such hybrid system have been proposed in recent years. We will focus on an extension of our work on the topic. (see our [relevant paper](https://link.springer.com/chapter/10.1007/978-3-030-58942-4_24))

We want to find the **maximum likelihood** assignement of decision variables. This means adding an objective function: a weighted sum over decision variables, using as weight log-probabilies provided by a neural network. 

But we cannot directly use our usual Sudoku decision variables to define this objective function, because of a mismatch between their domain ($\{1,\ldots, 9\}$) and machine learning classes $\in \{empty, 1, \ldots, 9\}$. Hence, we introduce a new set of *perception variables* that acts as an interface layer. 


Notable differences with the previous setting: 
- No given clues: we have to rely on (probabilistic) output of the machine learning model to infer the value (or the absence of) for each cell
- The machine learning model should learn to classify empty cells, while domains of our decision variables are [1, ..., 9]
 

In [ ]:
def get_visual_sudoku_full_image_model(ml_predictions, precision=1e-5):
    # base model and decision variables 
    visual_sudoku_problem = get_sudoku_model()
    model = visual_sudoku_problem['model']
    decision_variables = visual_sudoku_problem['variables']
    # introduce a layer of 'perception' variables, as an interface
    # between the solver and the ml network
    # their domain is [0, ..., 9], with 0 acting as the 'empty' symbol
    perception_variables = cp.intvar(0, 9, shape=decision_variables.shape, name='perception')

    # convert predictions to logspace
    logprobs = np.log(np.maximum( ml_predictions, precision ))
    # cp solver requires integer values
    logprobs = np.array(logprobs / precision).astype(int)
    # switch to cpm_array for more features
    logprobs = cp.cpm_array(logprobs)
    # build the objective function over perception variables 
    objective_function = sum(logprobs[idx][v] for idx,v in np.ndenumerate(perception_variables))
    model.maximize(objective_function)
    
    # channeling constraints to link decision variables to perception variables
    # perception variable is either 'empty' or matches grid symbol
    model+= [(perception_variables != 0).implies(decision_variables == perception_variables)]
    # keep track of perception variables as well 
    visual_sudoku_problem['perception'] = perception_variables
    return visual_sudoku_problem

def solve_visual_sudoku_full_image(visual_sudoku_problem, solver_params=dict()):
    model = visual_sudoku_problem['model']
    dvars, pvars = visual_sudoku_problem['variables'], visual_sudoku_problem['perception']
    s = CPM_ortools(model)
    if s.solve(**solver_params):
        results = {
            'solution':dvars.value(),
            'perception':pvars.value()
        }
    else:
        # in case of infeasibility, nan
        results = {
            'solution':np.full_like(dvars.value(), np.nan),
            'perception':np.full_like(dvars.value(), np.nan)
        }
    results['status'] = np.asarray(s.cpm_status.exitstatus.value)
    results['runtime'] = np.asarray(s.cpm_status.runtime)
    return results


## Pruning the search space

Adding a cost function turns the Sudoku into a constrained optimization problem. As the solver now reasons over probability distributions for all cells, we expect the solving of a visual sudoku instance to be slower. 

**Task: alleviate this issue by constraining the solver to only use the top-k best predictions for each cells**

*Tip: this could be easily implemented with a [cp.expressions.InDomain](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.InDomain) constraint*

In [ ]:
def post_topk_constraint(model, perception_variables, ml_predictions, k=9): 
    ...


## Recap: Convolutional Neural Network

In [ ]:
# These should match the values for your pre-trained CNN
hyperparams = {
    # preprocessing
    'size':(300,300),
    # architecture
    'dropout':0.25,
    'batchnorm':True,
    'pooling_kernel_size':3,
    'pooling_stride':3,
}

In [ ]:
from utils import make_conv_layers, make_fc_layers_global_pooling
import torchvision.transforms as T 
from torchvision.io import read_image

base_tsfm = T.Compose([
    T.Lambda(lambda img_str:read_image(img_str)),
    T.ToPILImage(),
    T.Resize(hyperparams['size']), # scale down image here to speed up training
    T.ToTensor(),
    T.Normalize( # Normalize the data (all values between -1 and 1) to improve convergence
        mean=[0.5320, 0.5209, 0.5204],
        std=[0.1862917, 0.19031495, 0.18998064]
    ), 
    #T.Grayscale(1) # grayscale the image
])

class FullImageCNN(nn.Module):
    """Generalized 5-layers CNN
    """

    def __init__(self, grid_shape=(9, 9), n_classes=10) -> None:
        super().__init__()
        self.grid_shape = grid_shape
        self.n_classes = n_classes
        
        # backbone with 5 convolutional layers
        # This should also match with layers in your pre-trained CNN
        full_image_backbone_config = [
            (32, 4, 2, 0),
            (64, 3, 2, 0), 
            (128, 3, 2, 0), 
            (256, 2, 2, 0), 
            (512, 2, 2, 0), 
        ]

        conv_layers = make_conv_layers(
            full_image_backbone_config, 
            in_channels=3, # number of channel in the input image (3 for RGB, 1 for grayscale)
            p=hyperparams['dropout'], # dropout rate
            pool_ks=hyperparams['pooling_kernel_size'], 
            pool_str=hyperparams['pooling_stride'], 
            batch_norm=hyperparams['batchnorm'] # controls whether to add batchnorm in-between convolutional layers
        )
        # because of the last convolutional layer, this backbone output a tensor whose first dimension is of size 512
        self.feat_extract = nn.Sequential( *conv_layers)

        # classifier
        out_layers = make_fc_layers_global_pooling(
            in_dim=512, # this should match with the size of first dimension of the previous layer (hence 512)
            out_shape=grid_shape, 
            num_classes=n_classes
        )
        self.classifier = nn.Sequential(*out_layers,  nn.Softmax(-1))
        # the output is of size 81 x 10 

    def forward(self, x):
        h = self.feat_extract(x)
        return {
            'predictions': self.classifier(h)
        }


In [ ]:
# Helper function to load weights of a pre-trained CNN
def load_from_checkpoint(version_id=0):
    cnn = FullImageCNN()
    dir_chkp_path = os.path.join('log', 'neural_network_p2', f'version_{version_id}', 'checkpoints')
    chkp_path = os.path.join(dir_chkp_path, os.listdir(dir_chkp_path)[0])
    CKPT_state_dict = torch.load(chkp_path)
    layer_names = list(cnn.state_dict().keys())
    to_load = OrderedDict(**{k.split('cnn.')[1]:v for k,v in CKPT_state_dict['state_dict'].items() if k.split('cnn.')[1] in layer_names})
    cnn.load_state_dict(to_load)
    return cnn.eval()

In [ ]:
pretrained_cnn = load_from_checkpoint(0)
pretrained_cnn

In [ ]:
# Helper function to get predictions from our pre-trained CNN
@torch.no_grad() # disable gradient computation
def get_predictions(cnn, image_preprocessed): 
    output = cnn(image_preprocessed.unsqueeze(0))['predictions']
    return output.detach().squeeze().numpy().reshape(9,9,-1)

In [ ]:
img_path = os.path.join('data', 'visual_sudoku', 'img', '059.jpg')
original = Image.open(img_path)

see_torch_image = T.ToPILImage()
image_preprocessed = base_tsfm(img_path)
ml_predictions = get_predictions(pretrained_cnn, image_preprocessed)

Even with a high accuracy (>95%), our CNN may still make prediction errors. Those errors may lead to an infeasible sudoku. 
This issue is directly addressed in the *visual sudoku solver*. Reasoning over probabilities enables the solver to sometimes correct neural network mistakes. 

In [ ]:
# visual sudoku cp model
visual_sudoku_problem = get_visual_sudoku_full_image_model(ml_predictions)
# top-k?
post_topk_constraint(visual_sudoku_problem['model'], visual_sudoku_problem['perception'], ml_predictions)
# solve 
results = solve_visual_sudoku_full_image(visual_sudoku_problem)


In [ ]:
# show solution
from utils import visu_sudoku
original = Image.open(img_path)
display(original)
visu_sudoku(results['solution'])

This solution looks good. However, it needs to be properly evaluated. To do so, it is enough to compare the value of cells containing starting clues, with values of the same cells in our solution. 



We can also observe values of perception variables: 

In [ ]:
visu_sudoku(results['perception'])

Compare it with the original to identify neural network mistakes. 

**Task: Evaluate the performance of your pre-trained neural network on the sudoku solving task.**

In [ ]:
# here is a loop over test instances 
test_set = [6, 35, 91, 96, 10, 80, 11, 72, 51, 20, 75, 82, 21, 4, 41, 14, 88, 56, 79, 94]
count_unsat = 0
count_solved = 0
for instance_id in test_set:
    img_path = os.path.join('data', 'visual_sudoku', 'img', f'{instance_id:03d}.jpg')
    label_path = os.path.join('data', 'visual_sudoku', 'label', f'{instance_id:03d}.npy')
    label = np.load(label_path)
    # ...

_________


# [Challenge] Higher-order knowledge exploitation

Because of the added objective function our visual Sudoku problem became an optimisation problem, which may have many feasible solutions.

However, any valid sudoku puzzle only admits a *unique solution* for a set of givens. To improve the efficiency of our hybrid approach, we can actually exploit this uniqueness property in the following manner:
- When the solver finds optimal solution, add the resulting assignment as a no-good (i.e. forbid it) and try to solve again
- if any feasible solution is found this way, previous assignment for given cells does not lead to unique solution

Iterate these steps until the uniqueness property is satisfied. 

In practice, we may need to loop several times depending on the accuracy of the classifier.

**Task implement the higher-order knowledge exploitation method. You can start from the helper code below**

*Tip: you can forbid an assignement of variables by doing* 
```python
model += [~cp.all((perception_variables ==  perception_variables.value())) ]
```

In [15]:
def solve_visual_sudoku_higher_order(visual_sudoku_problem, solver_params=dict(), max_iter=100):
    results = solve_visual_sudoku_full_image(visual_sudoku_problem, solver_params)
    #Write a loop repeating following steps:
    # while results['solution'] is not unique or iteration < max_iter:
    #   add nogood to the vizsudoku model
    #   solve again
    #   iteration += 1


______

# [Challenge] Printed and Handwritten digits (part 2)

As you may have notice, some images contains both printed and handwritten digits. 
We can assume that, as printed value genrally make up the starting clues of the puzzle, they are more reliable than handwritten values provided by a player.

<img src="data/visual_sudoku/img/030.jpg" alt="sudoku" style="width: 400px;"/>

A smart hybrid CP-ML solver could exploit this information to improve its rate of correctly solved instances. 

Assuming that your CNN provides for each cell: 
1. the probability distribution over possible values (as a 9x9x10 tensor)
2. the probability that it contains a printed digit (as a 9 x 9 matrix)

**Task: implement a new `get_visual_sudoku_full_image_model` that accepts two sets of ML predictions described above. There are many ways to integrate them into the model (through the objective function). Do you observe any gain in performance? 
